Package import.

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import random, seed
from copy import deepcopy
from queue import PriorityQueue

## Simple functions from the first classes
This is left just as an example (to compare the behaviour).

In [ ]:
def print_matrix(vertices, matrix):
  """
  Printing a graph given by adjacency matrix
  """
  n = len(matrix)
  if (vertices is not None) and (len(vertices) == n):
    vv = vertices
  else:
    vv = range(1, n+1)
  for i in range(n):
    print(vv[i], ":", end="")
    for j in range(n):
      if matrix[i, j]:
        print(" ", vv[j], end="")
    print("")

def print_dict(graph):
  """
  Printing of a graph (given as a dictionary/neighbouring list)
  """
  for v in graph:
    print(v, ":", end="")
    for u in graph[v]:
      print(" ", u, end="")
    print("")

## Class *Graph*

In [ ]:
class Graph:
    def __init__(self, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_matrix(cls, matrix, vertices = None):
        if (vertices is None) or (len(vertices) != len(matrix)):
            vertices = [*range(1, len(matrix) + 1)]
        return cls.from_dict(cls._matrix_to_dict(matrix, vertices))

    # two private methods matrix <-> dictionaries
    def _matrix_to_dict(matrix, vertices: list) -> dict:
        """
        Converts a graph given as an adjacency matrix to a graph in dict form.
        """
        res_dict = {}
        for i, v in enumerate(vertices):
            neighbours = [vertices[j] for j, edge in enumerate(matrix[i]) if edge]
            res_dict[v] = neighbours
        return res_dict

    def _dict_to_matrix(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to its adjacency matrix.
        """
        n = len(_dict)
        vertices = [*_dict.keys()]
        matrix = np.zeros(shape = (n, n), dtype=int)
        for u,v in [
            (vertices.index(u), vertices.index(v))
            for u, row in _dict.items() for v in row
        ]:
            matrix[u][v] += 1
        return matrix

    def vertices(self) -> list:
        """
        Returns list of vertices of the graph.
        """
        return [*self.graph.keys()]

    def matrix(self) -> np.array:
        """
        Returns the adjacency matrix of the graph.
        """
        return self._dict_to_matrix(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" {u}"
            res += "\n"
        return res

    def plot(self):
      """
      Plots the graph using networkx package.
      """
      G = nx.Graph(self.graph)
      pos = nx.spring_layout(G)
      nx.draw(G, pos, with_labels=True)
      plt.show()


    # The following is for free thanks to the above
    def to_neighbourlist(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.\n
        Filename is a file path.
        """
        file = open(filename, "w")  # open textfile for writing
        file.write(str(self))
        file.close()

    # Modyfying graphs
    def add_vertex(self, vertex):
        """
        Adds a new vertex to the graph.
        """
        if vertex not in self.graph:
            self.graph[vertex] = []

    def del_vertex(self, vertex):
        """
        Removes a vertex from the graph.
        """
        if vertex in self.graph:
            self.graph.pop(vertex)
            for u in self.graph:
                if vertex in self.graph[u]:
                    self.graph[u].remove(vertex)

    def add_arc(self, arc):
        """
        Given pair of vertices (arc variable) add an arc to the graph
        We consider simple, directed graphs.
        """
        u, v = arc
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)

    def add_edge(self, edge: list):
        """
        Given pair of vertices (edge variable) add an edge to existing graph.
        We consider simple, undirected graphs, as symmetric digraphs without loops.
        """
        u, v = edge
        if u == v:
            raise ValueError("Loops are not allowed!")
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    # reading from a file
    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Read the graph from file, that in each line contains either
        the description of a vertex (one word) or
        the description of an edge/arc (at least 2 words).
        The resulting graph is returned as a neighbourhood list.
        Variable "filename" contains the whole path to the file.
        """
        graph = Graph()
        file = open(filename, "r")          # open the file to read
        for line in file:                   # for each line of the file
          words = line.strip().split()      # splits the line into words
          if len(words) == 1:               # one word - vertex description
            graph.add_vertex(words[0])
          elif len(words) >= 2:             # at least two words, first two are the edge description
            if directed:
              graph.add_arc([words[0], words[1]])
            else:
              graph.add_edge([words[0], words[1]])
        file.close()
        return graph

    @staticmethod
    def random_graph(n: int, p: float):
        """
        Creates a random graph in G(n, p) model.
        """
        rand_graph = Graph()
        for i in range(1, n + 1):
            rand_graph.add_vertex(i)
            for j in range(1, i):
                if random() < p:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def random_bipartite_graph(m, n, p):
        """
        Creates a random bipartite graph G(m, n, p) model.
        m: number of vertices in the first partition.
        n: number of vertices in the second partition.
        p: probability of an edge between vertices from different partitions.
        """
        bipartite_graph = Graph()
        for i in range(1, m + n + 1):
            bipartite_graph.add_vertex(i)

        for u in range(1, m + 1):
            for w in range(m + 1, m + n + 1):
                if random() < p:
                    bipartite_graph.add_edge([u, w])
        return bipartite_graph

    @staticmethod
    def cycle(n: int):
        """
        Creates a cycle C_n on n vertices
        """
        cycle = Graph()
        for i in range(n-1):
          cycle.add_edge([i+1, i+2])
        cycle.add_edge([1, n])
        return cycle


    def Prufer(self):
      """
      Returns the Prufer code of a tree.
      It is necessary that the graph is a tree (it is not checked).
      Result is given as a string (empty for trees on 1 or 2 vertices).
      """
      tr = deepcopy(self.graph)   # copy of a tree, as we destroy it
      code = ""
      for i in range(len(self.graph) - 2):
        for x in sorted(tr):
          if len(tr[x]) == 1:   # least leaf
            break
        v = tr[x][0]            # the unique neighbour of x
        code = code + f"{v} "
        tr[v].remove(x)         # remove x from neighbours of v
        tr.pop(x)               # remove x from the tree
      return code.strip()

    @staticmethod
    def tree_from_Prufer(code: str):
        """
        Creating a tree from a Prufer code.
        """
        tree = Graph()
        clist = [int(x) for x in code.strip().split()]   # code as a list of numbers
        n = len(clist) + 2                  # number of vertices
        vert = [*range(1, n+1)]             # list of numbers 1..n
        for v in vert:
          tree.add_vertex(v)
        for i in range(n-2):
          for x in vert:
            if not x in clist:    # x - least leaf
              break
          v = clist.pop(0)    # remove the first element from the code - the neighbour of x
          tree.add_edge((x, v))
          vert.remove(x)
        tree.add_edge(vert)
        return tree


    def connected_components(self):
      """
      Looks for connected components of undirected graph.
      Returns a list of its vertex-sets.
      Remark: the first element contains the set of all graph vertices
      """
      def DFS(u):
        """
        Deep first search (as internal method).
        """
        for w in self.graph[u]:
          if w not in VT[0]:      # w - not visited yet
            VT[0].add(w)          # already visited
            VT[-1].add(w)         # w - in the last connected component
            DFS(w)

      """
      VT - list of vertex sets VT[i] for i > 0 - is a vertex set of i-th connected component
      VT[0] - is a vertex set of the spanning forest (or during the algorithm list of visited vertices).
      """
      VT = [set([])]
      for v in self.graph:
        if v not in VT[0]:      # v is not visited
          VT[0].add(v)
          VT.append(set([v]))   # statring point of new conected component
          DFS(v)
      return VT

    def preorder(self, v, visited=None):
        """
        Performs a pre-order traversal starting from vertex v and prints the vertices.
        """
        if visited is None:
            visited = set()

        if v not in visited:
            print(v, end=" ")
            visited.add(v)
            for neighbor in self.graph.get(v, []):
                self.preorder(neighbor, visited)

    def postorder(self, v, visited=None):
        """
        Performs a post-order traversal starting from vertex v and prints the vertices.
        """
        if visited is None:
            visited = set()

        if v not in visited:
            visited.add(v)
            for neighbor in self.graph.get(v, []):
                self.postorder(neighbor, visited)
            print(v, end=" ")

    def InducedSubgraph(self, V):
        """
        Returns a subgraph induced by the vertex set V.
        """
        subgraph = Graph()
        for v in V:
            subgraph.add_vertex(v)
            for w in self.graph[v]:
                if w in V:
                    subgraph.add_arc([v, w])
        return subgraph


    def ConnectedComponentsGraphs(self):
        """
        Returns a list of graphs, where each graph represents a connected component.
        Uses the existing connected_components method to find vertex sets.
        """
        components_vertex_sets = self.connected_components()
        list_of_component_graphs = []

        for component_set in components_vertex_sets[1:]:
            component_graph = Graph()
            for u in component_set:
                #component_graph.add_vertex(u)              # more general and elegant
                #for v in self.graph[u]:
                #  component_graph.add_arc([u, v])
                component_graph.graph[u] = self.graph[u]    # simply does the same stuff
            list_of_component_graphs.append(component_graph)

        return list_of_component_graphs

    def distance(self, v):
      """
      Computes the distance from vertex v to all other reachable vertices in the graph.
      It uses the BFS approach.
      Result is given as a dictionary of distances.
      """
      dist = {v: 0}     # starting point of a dictionary
      queue = [v]
      while len(queue) > 0:
        u = queue.pop(0)
        for w in self.graph[u]:
          if w not in dist:     # w not visited already
            dist[w] = dist[u] + 1
            queue.append(w)
      return dist




      #________________________________________
      ### NEW TASKS ADDED below

    def StronglyConnectedComponents(self):
        """
        Returns a list of strongly connected components of a directed graph.
        Uses topological sorting on the reverse graph (Kosaraju algorithm).
        """
        graph = self.graph

        # First DFS (finishing order)
        visited = set()
        order = []

        def dfs1(v):
            visited.add(v)
            for w in graph.get(v, []):
                if w not in visited:
                    dfs1(w)
            order.append(v)

        for v in graph:
            if v not in visited:
                dfs1(v)

        # Reverse graph
        rev = {v: [] for v in graph}
        for u in graph:
            for v in graph[u]:
                rev[v].append(u)

        # Second DFS on reversed graph
        visited = set()
        components = []

        def dfs2(v, comp):
            visited.add(v)
            comp.append(v)
            for w in rev.get(v, []):
                if w not in visited:
                    dfs2(w, comp)

        # process vertices in decreasing finish time
        for v in reversed(order):
            if v not in visited:
                comp = []
                dfs2(v, comp)
                components.append(comp)

        return components

        ## EX 2 and 3 added to WeightedGraph


## Class *WeightedGraph*

In [ ]:
class WeightedGraph(Graph):
    def __init__(self, graph=None, weights=None):
        if weights is None:
            weights = {}
        self.weights = weights
        super().__init__(graph)

    #redefinition of printing and plotting
    def __str__(self):
        res = ""
        #res = super().__str__()
        #res += '-------------------------\n'
        for v in self.graph:
            res += f'{v}: '
            for u in self.graph[v]:
                res += f'{u}({self.weights[(v, u)]}) '
            res += '\n'
        return res

    def plot(self):
      """
      Plots the graph using networkx package.
      """
      G = nx.Graph(self.graph)
      nx.set_edge_attributes(G, self.weights, 'weight')
      pos = nx.spring_layout(G)
      nx.draw(G, pos, with_labels=True, node_color='lightblue')
      edge_labels = nx.get_edge_attributes(G, 'weight')
      nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
      plt.show()

    # Modyfying graphs
    def del_vertex(self, v):
      """
      Removes a vertex from the graph.
      """
      # TODO - should be done in a different way
      super().del_vertex(v)

    def add_arc(self, arc, weight = 1):
       """
       Given pair of vertices (arc variable) add an arc to the graph
       We consider simple, directed graphs.
       """
       super().add_arc(arc)
       u, v = arc
       self.weights[(u, v)] = weight

    def add_edge(self, edge, weight = 1):
       """
       Given pair of vertices (edge variable) add an edge to existing graph.
       We consider simple, undirected graphs, as symmetric digraphs without loops.
       """
       super().add_edge(edge)
       u, v = edge
       self.weights[(u, v)] = weight
       self.weights[(v, u)] = weight

    # reading from a file
    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Read the graph from file, that in each line contains either
        the description of a vertex (one word) or
        the description of an edge/arc (at least 2 words).
        When we have at least 3 words we assume the third one is an edge weight (of type int).
        The resulting graph is returned as a neighbourhood list.
        Variable "filename" contains the whole path to the file.
        """
        graph = WeightedGraph()
        file = open(filename, "r")          # open the file to read
        for line in file:                   # for each line of the file
          words = line.strip().split()
          if len(words) == 1:               # one word - vertex description
            graph.add_vertex(words[0])
          elif len(words) == 2:             # exactly two words, unweghted edge description
            if directed:
              graph.add_arc([words[0], words[1]])
            else:
              graph.add_edge([words[0], words[1]])
          elif len(words) >= 3:             # at least three words, weighted edge description
            if directed:
              graph.add_arc([words[0], words[1]], int(words[2]))
            else:
              graph.add_edge([words[0], words[1]], int(words[2]))
        file.close()
        return graph

    def min_spanning_tree(self):
      """
      Computes a minimum spanning tree using Jarnik - Prim's algorithm.
      """
      for v in self.graph:    # choose any vertex of a graph
        break
      wtree = WeightedGraph()
      wtree.add_vertex(v)
      total_weight = 0
      q = PriorityQueue()
      for w in self.graph[v]:
        q.put((self.weights[(v, w)], v, w))
      while not q.empty():
        weight, u, v = q.get()
        if v not in wtree.graph:
          wtree.add_edge([u, v], weight)
          total_weight += weight
          for w in self.graph[v]:
            if w not in wtree.graph:
              q.put((self.weights[(v, w)], v, w))
      if len(wtree.graph) != len(self.graph):
        print("The graph is not connected! Returned is a spanning tree of some component.")
      return total_weight, wtree
    ####
    ###_______________________________
    ### Added EX 2
    def to_weight_matrix(self):
        """
        Converts the weighted graph to an adjacency matrix of weights.
        Non-edges get weight = +inf.
        """
        verts = self.vertices()
        n = len(verts)
        M = np.full((n, n), float('inf'))

        # distance to itself = 0
        for i in range(n):
            M[i, i] = 0

        for (u, v), w in self.weights.items():
            i = verts.index(u)
            j = verts.index(v)
            M[i, j] = w

        return verts, M

    @classmethod
    def from_weight_matrix(cls, vertices, matrix):
        """
        Creates a WeightedGraph from a given weight matrix.
        """
        WG = WeightedGraph()
        n = len(vertices)

        for v in vertices:
            WG.add_vertex(v)

        for i in range(n):
            for j in range(n):
                if i != j and matrix[i, j] != float('inf'):
                    WG.add_arc([vertices[i], vertices[j]], matrix[i, j])

        return WG

    ## EX 3
    def FloydWarshall(self):
        """
        Applies the Floyd–Warshall algorithm.
        Returns: (vertices, distance_matrix)
        """
        verts, M = self.to_weight_matrix()
        n = len(verts)

        dist = M.copy()

        for k in range(n):
            for i in range(n):
                for j in range(n):
                    if dist[i, k] + dist[k, j] < dist[i, j]:
                        dist[i, j] = dist[i, k] + dist[k, j]

        return verts, dist

# 1. Write function StronglyConnectedComponents that returns a list of digraph's strongly connected components. Hint: it can use topological sorting a s a tool.


In [ ]:
def StronglyConnectedComponents(self):
    """
    Returns a list of strongly connected components of a directed graph.
    Uses topological sorting on the reverse graph (Kosaraju algorithm).
    """
    graph = self.graph

    # First DFS (finishing order)
    visited = set()
    order = []

    def dfs1(v):
        visited.add(v)
        for w in graph.get(v, []):
            if w not in visited:
                dfs1(w)
        order.append(v)
    for v in graph:
        if v not in visited:
            dfs1(v)

    # Reverse graph
    rev = {v: [] for v in graph}
    for u in graph:
        for v in graph[u]:
            rev[v].append(u)

    # Second DFS on reversed graph
    visited = set()
    components = []

    def dfs2(v, comp):
        visited.add(v)
        comp.append(v)
        for w in rev.get(v, []):
            if w not in visited:
                dfs2(w, comp)

    # process vertices in decreasing finish time
    for v in reversed(order):
        if v not in visited:
            comp = []
            dfs2(v, comp)
            components.append(comp)

    return components

In [ ]:
%%writefile edges_scc.txt
a b
b c
c a
d e
e d
f

Overwriting edges_scc.txt


In [ ]:
G = Graph.from_edges("edges_scc.txt", directed=1)
print(G)
print("SCC:", G.StronglyConnectedComponents())


a: b
b: c
c: a
d: e
e: d
f:

SCC: [['f'], ['d', 'e'], ['a', 'c', 'b']]


# 2. For a weighted graph: create functions that converts graph to/from a matrix of weights.


In [ ]:
def to_weight_matrix(self):
    """
    Converts the weighted graph to an adjacency matrix of weights.
    Non-edges get weight = +inf.
    """
    verts = self.vertices()
    n = len(verts)
    M = np.full((n, n), float('inf'))

    # distance to itself = 0
    for i in range(n):
        M[i, i] = 0

    for (u, v), w in self.weights.items():
        i = verts.index(u)
        j = verts.index(v)
        M[i, j] = w

        return verts, M

@classmethod
def from_weight_matrix(cls, vertices, matrix):
    """
    Creates a WeightedGraph from a given weight matrix.
    """
    WG = WeightedGraph()
    n = len(vertices)

    for v in vertices:
        WG.add_vertex(v)

    for i in range(n):
        for j in range(n):
            if i != j and matrix[i, j] != float('inf'):
                WG.add_arc([vertices[i], vertices[j]], matrix[i, j])

    return WG

In [ ]:
%%writefile edges_weighted.txt
a b 3
a c 8
b d 2
c d 1
d e 4
e a 7

Overwriting edges_weighted.txt


In [ ]:
##Usage
WG = WeightedGraph.from_edges("edges_weighted.txt", directed=1)

verts, M = WG.to_weight_matrix()
print("Vertices:", verts)
print("Weight matrix:\n", M)

WG2 = WeightedGraph.from_weight_matrix(verts, M)
print("\nGraph from matrix:")
print(WG2)


Vertices: ['a', 'b', 'c', 'd', 'e']
Weight matrix:
 [[ 0.  3.  8. inf inf]
 [inf  0. inf  2. inf]
 [inf inf  0.  1. inf]
 [inf inf inf  0.  4.]
 [ 7. inf inf inf  0.]]

Graph from matrix:
a: b(3.0) c(8.0) 
b: d(2.0) 
c: d(1.0) 
d: e(4.0) 
e: a(7.0) 



# 3. Apply Floyd-Warshall algorithm to compute distance matrix for all vertices of a graph. Hint: use a function from the above task to obtain a matrix from a graph.

In [ ]:
## EX 3

def FloydWarshall(self):
  """
    Applies the Floyd–Warshall algorithm.
    Returns: (vertices, distance_matrix)
  """
  verts, M = self.to_weight_matrix()
  n = len(verts)
  dist = M.copy()
  for k in range(n):
      for i in range(n):
          for j in range(n):
              if dist[i, k] + dist[k, j] < dist[i, j]:
                  dist[i, j] = dist[i, k] + dist[k, j]
  return verts, dist

In [ ]:
%%writefile edges_fw.txt
a b 4
a c 1
c b 2
b d 5
c d 8
d e 3
e b -6

Overwriting edges_fw.txt


In [ ]:
# Usage
WG = WeightedGraph.from_edges("edges_fw.txt", directed=1)
verts, D = WG.FloydWarshall()

print("Vertices:", verts)
print("Distance matrix:")
print(D)

Vertices: ['a', 'b', 'c', 'd', 'e']
Distance matrix:
[[ 0.  3.  1.  8. 11.]
 [inf  0. inf  5.  8.]
 [inf  2.  0.  7. 10.]
 [inf -3. inf  0.  3.]
 [inf -6. inf -1.  0.]]
